In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from config import g_key

In [67]:
city_df = pd.read_csv('city_data.csv')
gmaps.configure(api_key=g_key)

city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,krasnyy yar,94,RU,1642701785,88,46.5331,48.3456,15.73,6.08
1,butaritari,79,KI,1642701209,71,3.0707,172.7902,81.66,17.43
2,camacha,20,PT,1642701785,55,33.0833,-16.3333,63.36,24.16
3,port hardy,100,CA,1642701291,93,50.6996,-127.4199,44.76,10.36
4,ushuaia,1,AR,1642701100,33,-54.8000,-68.3000,67.62,4.43


In [68]:
locations = city_df[['Lat', 'Lng']]
weights = city_df['Humidity'].astype(float)

In [69]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity = 300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [70]:
weather_df = city_df.loc[(city_df['Max_Temp'] > 70) & (city_df['Max_Temp'] < 80) & (city_df['Wind_Speed'] < 10) & (city_df['Cloudiness'] == 0)].dropna()

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
72,east london,0,ZA,1642701421,78,-33.0153,27.9116,79.74,8.05
87,cape town,0,ZA,1642701106,64,-33.9258,18.4232,70.21,3.00
125,taoudenni,0,ML,1642701805,14,22.6783,-3.9836,70.61,8.68
136,kailua,0,US,1642701807,85,21.4022,-157.7394,75.18,0.00
197,koungou,0,YT,1642701820,88,-12.7336,45.2042,76.71,5.75


In [71]:
hotel_df = weather_df

hotel_df['Hotel Name'] = ''

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
72,east london,0,ZA,1642701421,78,-33.0153,27.9116,79.74,8.05,
87,cape town,0,ZA,1642701106,64,-33.9258,18.4232,70.21,3.00,
125,taoudenni,0,ML,1642701805,14,22.6783,-3.9836,70.61,8.68,
136,kailua,0,US,1642701807,85,21.4022,-157.7394,75.18,0.00,
197,koungou,0,YT,1642701820,88,-12.7336,45.2042,76.71,5.75,


In [72]:

locations = hotel_df[['Lat', 'Lng']]
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {
    "type": "hotel",
    "keyword": "hotel",
    "radius": 50000,
    "key": g_key
}

for index, location in locations.iterrows():
    location_ = f'{location[0]},{location[1]}'
    params["location"] = location_
    print('Retrieveing Data for index: {0}'.format(index))
    
    response = requests.get(url, params=params)
    locations_json = response.json()
    try:
        hotel_name = locations_json['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        
        
    except:
        hotel_df.loc[index, 'Hotel Name'] = np.NAN
        print("Could not Parse Data")

Retrieveing Data for index: 72
Retrieveing Data for index: 87
Retrieveing Data for index: 125
Could not Parse Data
Retrieveing Data for index: 136
Retrieveing Data for index: 197
Retrieveing Data for index: 243
Retrieveing Data for index: 260
Retrieveing Data for index: 304
Retrieveing Data for index: 345
Retrieveing Data for index: 347
Retrieveing Data for index: 380
Retrieveing Data for index: 447
Retrieveing Data for index: 489


In [73]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
72,east london,0,ZA,1642701421,78,-33.0153,27.9116,79.74,8.05,Blue Lagoon Hotel
87,cape town,0,ZA,1642701106,64,-33.9258,18.4232,70.21,3.00,12 Apostles Hotel & Spa
125,taoudenni,0,ML,1642701805,14,22.6783,-3.9836,70.61,8.68,NaN
136,kailua,0,US,1642701807,85,21.4022,-157.7394,75.18,0.00,Halekulani Hotel
197,koungou,0,YT,1642701820,88,-12.7336,45.2042,76.71,5.75,Hotel Maharajah Mayotte


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [ info_box_template.format(**row) for index, row in hotel_df.iterrows()]

['\n<dl>\n<dt>Name</dt><dd>Blue Lagoon Hotel</dd>\n<dt>City</dt><dd>east london</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>12 Apostles Hotel & Spa</dd>\n<dt>City</dt><dd>cape town</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>nan</dd>\n<dt>City</dt><dd>taoudenni</dd>\n<dt>Country</dt><dd>ML</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Halekulani Hotel</dd>\n<dt>City</dt><dd>kailua</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Maharajah Mayotte</dd>\n<dt>City</dt><dd>koungou</dd>\n<dt>Country</dt><dd>YT</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>The Dwarika Hotel</dd>\n<dt>City</dt><dd>dwarka</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>JP Resort</dd>\n<dt>City</dt><dd>naliya</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Marina Del Rey Hotel</dd>\n<dt>City</dt><dd>malibu</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Halekulan

In [75]:
symbol_layer = gmaps.marker_layer(
        locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)

fig

Figure(layout=FigureLayout(height='420px'))